In [3]:
import pandas as pd
import utils

In [4]:
# Plot candle price
pair ="EUR_USD"
granularity = "H1"

In [5]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [8]:
df.describe() #show for numerical values

,volume
count,5000.000000
mean,3969.037600
std,2928.376527
min,125.000000
25%,2091.500000
50%,3270.500000
75%,4989.250000
max,27524.000000


In [9]:
non_cols = ['time', 'volume']

In [10]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [13]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [15]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,3969.037600,1.081033,1.081717,1.080359,1.081030,1.080941,1.081635,1.080268,1.080947,1.081125,1.081806,1.080441,1.081112
std,2928.376527,0.017015,0.017010,0.017015,0.017017,0.017014,0.017010,0.017014,0.017016,0.017017,0.017010,0.017016,0.017018
min,125.000000,1.045260,1.046640,1.044850,1.045270,1.045180,1.046550,1.044770,1.045200,1.045330,1.046720,1.044930,1.045340
25%,2091.500000,1.067720,1.068535,1.066915,1.067708,1.067638,1.068458,1.066828,1.067630,1.067798,1.068610,1.066988,1.067780
50%,3270.500000,1.081910,1.082780,1.081075,1.081900,1.081840,1.082705,1.080980,1.081830,1.081985,1.082865,1.081155,1.081970
75%,4989.250000,1.093400,1.094065,1.092600,1.093405,1.093332,1.093993,1.092522,1.093335,1.093472,1.094148,1.092672,1.093485
max,27524.000000,1.125740,1.127570,1.125050,1.125760,1.125650,1.127490,1.124960,1.125670,1.125820,1.127650,1.125130,1.125840
